# Hands-on Python Excersises:

- ***Tutorial 0 : Getting Started with Movingpandas***
- Tutorial 1 : Trajectory Preprocessing
- Tutorial 2 : Taxi Trajectory Dashboard Visualization



# Tutorial 0 : Getting Started with MovingPandas


MovingPandas provides a trajectory datatype based on GeoPandas. The project home is at https://github.com/movingpandas/movingpandas

The documentation is available at https://movingpandas.readthedocs.io/

```
! pip install movingpandas

! pip install cartopy

! pip install geoviews

! pip install mapclassify

! pip install keplergl
```


In [ ]:
# ! pip install movingpandas
# ! pip install cartopy
# ! pip install geoviews
# ! pip install mapclassify
# ! pip install keplergl

In [ ]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import shapely as shp
import matplotlib.pyplot as plt

import folium

from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta


import warnings

warnings.filterwarnings("ignore")


In [ ]:
mpd.show_versions()

## Creating a trajectory from scratch


Trajectory objects consist of trajectory ID and a GeoPandas GeoDataFrame with a DateTimeindex. At minimum we need a DataFrame with Geometry and Time information.

Let's create a small toy trajectory to see how this works:

In [ ]:

# Lets create a dataframe with 4 points, each with a timestamp
df = pd.DataFrame(
    [
        {"time": datetime(2025, 1, 1, 12, 0, 0), "x": 0, "y": 0},
        {"time": datetime(2025, 1, 1, 12, 6, 0), "x": 48, "y": 0},
        {"time": datetime(2025, 1, 1, 12, 10, 0), "x": 48, "y": 48},
        {"time": datetime(2025, 1, 1, 12, 15, 0), "x": 100, "y": 100},
    ]
)

df

In [ ]:
# or we can directly create trajectory by defining the x and y columns
toy_traj = mpd.Trajectory(df, traj_id=1, x='x', y='y', t='time', crs=31256)
print(toy_traj)

In [ ]:
toy_traj.df

## Processing trajectories

We can **compute the distance, speed, and acceleration** of movement along the trajectory (between consecutive points). The default distance units are **meters** (or **CRS units**, if the CRS units are not known or specified), and the default time units are **seconds**:

In [ ]:
# add distance to the trajectory
toy_traj.add_distance(overwrite=True).df

In [ ]:
toy_traj.add_timedelta(overwrite=True).df

In [ ]:
# add speed to the trajectory
toy_traj.add_speed(units=("km", "h"), overwrite=True).df

In [ ]:
# add acceleration to the trajectory
toy_traj.add_acceleration(overwrite=True, units=("km", "h", "s")).df

If you want to use different units, you can specify them. Allowed units include metric units from mm to km, imperial units from inch to mile, nautical miles, and non-standard units which are used as CRS distance units e.g. US Survey units.

In [ ]:
toy_traj.add_distance(overwrite=True, name="distance (km)", units="km")
toy_traj.add_distance(overwrite=True, name="distance (yards)", units="yd")
toy_traj.add_speed(overwrite=True, name="speed (ft/min)", units=("ft", "min"))
toy_traj.add_speed(overwrite=True, name="speed (knots)", units=("nm", "h"))
toy_traj.add_acceleration(
    overwrite=True, name="acceleration (mph/s)", units=("mi", "h", "s")
)
toy_traj.df

## Visualizing trajectories

To **visualize the trajectory**, we can turn it into a linestring.

(The notebook environment automatically plots Shapely geometry objects like the LineString returned by to_linestring().)

In [ ]:
toy_traj.to_linestring()

We can also visualize the speed values:

In [ ]:
toy_traj.plot(column="speed", linewidth=5, capstyle="round", legend=True)

In contrast to the earlier example where we visualized the whole trajectory as one linestring, the trajectory plot() function draws each line segment individually and thus each can have a different color.

In [ ]:

toy_traj.explore(column="speed", legend=True)

## Analyzing trajectories

### Extracting a moving object's position at a certain time

For example, let's have a look at the get_position_at() function:

In [ ]:
toy_traj.get_position_at(datetime(2025, 1, 1, 12, 6, 0), method="nearest")

To see its coordinates, we can look at the print output:

In [ ]:
print(toy_traj.get_position_at(datetime(2025, 1, 1, 12, 6, 0), method="nearest"))

If the timestamp falls outside the time range between trajectory start and end time, we get an error:

In [ ]:
print(toy_traj.get_position_at(datetime(2024, 1, 1, 12, 6, 0), method="nearest"))

## Beyond toy trajectories: Loading trajectories from CSV



We are going to work with Taxi Trajectory data: A dataset describing a complete year (from 01/07/2013 to 30/06/2014) of the trajectories for all the 442 taxis running in the city of Porto, in Portugal

In [ ]:
path = "/content/drive/MyDrive/i4Geo_Workshop/new_taxi_porto.csv"
taxi_data = pd.read_csv(path)
taxi_data

In [ ]:
# filter the data to only include the first 1000 rows

taxi_data = taxi_data.head(5000)

taxi_data

After reading the trajectory point data from file, we want to construct the trajectories.

### Creating trajectories with TrajectoryCollection

TrajectoryCollection is a convenience class that takes care of creating trajectories from a GeoDataFrame:

In [ ]:

traj_collection = mpd.TrajectoryCollection(taxi_data, "TRIP_ID", t="datetime", y="latitude", x="longitude", crs=4326)
print(traj_collection)

In [ ]:
traj_collection.plot(column="TRIP_ID", legend=True, figsize=(9, 5))

In [ ]:
traj_collection.explore(column="TRIP_ID", legend=True, style_kwds={'weight': 5})

In [ ]:
# create trajectory collection based on TAXI_ID
traj_collection_taxi= mpd.TrajectoryCollection(taxi_data, "TAXI_ID", t="datetime", y="latitude", x="longitude", crs=4326)
print(traj_collection_taxi)

In [ ]:
traj_collection_taxi.explore(
    column="TAXI_ID",
    legend=True,
    style_kwds={'weight': 5},
    cmap="Set1")

### Converting TrajectoryCollections back to GeoDataFrames


In [ ]:
traj_collection.to_point_gdf().head(3)

Useful when want to visualize data in ArcGIS, or QGIS

In [ ]:
traj_collection.to_line_gdf().head(3)

In [ ]:
traj_collection.to_traj_gdf(wkt=True).head(3)

In [ ]:
traj_collection.to_traj_gdf(wkt=True).to_csv("taxi_porto_traj_line.csv")

### Wroking with individual trajectory in trajectory collection

In [ ]:
my_traj = traj_collection.trajectories[1]
print(my_traj)

In [ ]:
m = my_traj.explore(style_kwds={'weight': 10}, name="Trajectory 1")
my_traj.df.explore(style_kwds={'weight': 10, 'color': 'red'}, m=m, name="Trajectory 1 points")
#add a layer control to the map
folium.LayerControl().add_to(m)
m

#m.save("my_traj.html")

Even if the TrajectoryCollection GeoDataFrame does not contain a speed column, we can still plot movement speed:

In [ ]:
my_traj.explore(column='speed', style_kwds={'weight': 15})